In [ ]:
# Load the PDF
pdf_path = "C:/Users/Sarthak/Downloads/harrison’s-principles-of-internal-medicine-21st-edition.pdf"
loader = PyPDFLoader(pdf_path)

# Extract data (list of Document objects)
extracted_data = loader.load()

# Split the extracted documents into chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

# Apply text splitting
text_chunks = text_split(extracted_data)

# Print a few chunks to verify
print(text_chunks[:2])  # Print first two chunks

In [ ]:
# Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [ ]:
embeddings = download_hugging_face_embeddings()

In [ ]:

import os
from dotenv import load_dotenv
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
print(PINECONE_API_KEY)  # Ensure it's set correctly

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define Index Name
index_name = "medbot"


# Check if Index Exists
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"Created new index: {index_name}")
else:
    print(f"Index '{index_name}' already exists.")

In [ ]:
from langchain.vectorstores import Pinecone

docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)
